In [3]:
:load_config

## Load Data

In [4]:
let di_reader: DiReader<usize> = DiReader {
    t: 0,
    o: 1,
    h: 2,
    l: 3,
    c: 4,
    v: 5,
    t_format: None,
    has_header: true,
};
let di: Di = di_reader.read_csv("kline_data.csv").await.to_di(aler, rl5m.tri_box());

let tick_reader: TickReader<usize> = TickReader {
    t: 0,
    c: 1,
    v: 2,
    ask1: 3,
    bid1: 4,
    ask1_v: 5,
    bid1_v: 6,
    t_format: None,
    has_header:true,
};
let tick_data: Vec<TickData> = tick_reader.read_csv("tick_data.csv").await;

## Build a strategy</br>
### There are many ways to build a strategy, so that when I'm doing it, I can focus on what I care.</br>

### **1. Build a strategy based on kline input and return a target position**

In [5]:
// a simple double moving average strategy
#[ta_derive2]
pub struct TwoMaStra {
    pub short_period: usize,
    pub long_period: usize,
}

#[typetag::serde]
impl Ktn for TwoMaStra {
    fn ktn(&self,_di: &Di) -> RetFnKtn {
        let mut last_norm_hold = NormHold::No;
        let mut short_ma = SMA::new(self.short_period).unwrap();
        let mut long_ma = SMA::new(self.long_period).unwrap();
        let mut last_short_value = 0f64;
        let mut last_long_value = 0f64;
        Box::new(move |di_kline| {
            let c = di_kline.di.c()[di_kline.i] as f64;
            let short_value = short_ma.next(c);
            let long_value = long_ma.next(c);
            match last_norm_hold {
                NormHold::No if di_kline.i != 0 => {
                    if last_short_value < last_long_value && short_value >= long_value {
                        last_norm_hold = NormHold::Lo(1.);
                    }
                }
                NormHold::Lo(_) if short_value < long_value => {
                    last_norm_hold = NormHold::No;
                }
                _ => {}
            }
            last_short_value = short_value;
            last_long_value = long_value;
            last_norm_hold.clone()
        })
    }
}

let two_ma_stra = TwoMaStra { short_period: 10, long_period: 20 };

In [6]:
// backtest the strategy with kline input, because it just backtest on klines, so the order matched price is just set
// to the kline close price, so the slippage must be set. 
let two_ma_stra_ptm: Ptm = two_ma_stra.ktn_box().to_ktn().to_ptm();
let pnl_res_dt: PnlRes<dt> = two_ma_stra_ptm.bt_kline((&di, cs2));

In [7]:
// pnl_res_dt.to_csv("pnl_res_dt.csv") // save the pnl to csv
pnl_res_dt.da().plot() // plot the pnl

<polyline fill="none" opacity="1" stroke="#FFFFFF" stroke-width="1" points="75,246 75,247 75,249 75,249 75,248 76,249 76,248 76,251 76,248 77,248 77,249 77,249 77,249 77,249 78,249 78,249 78,249 78,249 79,249 79,249 79,249 79,245 79,244 80,244 80,243 80,244 80,244 81,244 81,245 81,243 81,244 81,244 82,244 82,244 82,245 82,244 83,245 83,246 83,246 83,246 83,246 84,246 84,246 84,243 84,242 85,242 85,242 85,241 85,238 85,238 86,239 86,239 86,240 86,240 87,240 87,240 87,239 87,240 87,240 88,240 88,240 88,241 88,239 89,239 89,240 89,240 89,240 89,239 90,238 90,238 90,239 90,238 91,238 91,238 91,238 91,238 91,238 92,238 92,238 92,238 92,238 93,238 93,238 93,238 93,240 93,240 94,242 94,242 94,242 94,242 95,243 95,242 95,242 95,242 96,243 96,243 96,243 96,243 96,242 97,243 97,243 97,243 97,243 98,243 98,244 98,244 98,244 98,244 99,245 99,245 99,245 99,245 100,245 100,244 100,243 100,245 100,245 101,245 101,245 101,247 101,246 102,246 102,248 102,249 102,249 102,246 103,245 103,246 103,247 103,248 104,247 104,247 104,248 104,248 104,250 105,249 105,251 105,251 105,251 106,252 106,252 106,252 106,252 106,252 107,253 107,254 107,254 107,254 108,252 108,252 108,252 108,253 108,253 109,253 109,253 109,252 109,252 110,253 110,253 110,255 110,255 110,254 111,255 111,255 111,255 111,254 112,255 112,252 112,253 112,252 112,252 113,253 113,253 113,253 113,252 114,252 114,251 114,253 114,253 115,253 115,253 115,254 115,253 115,256 116,255 116,255 116,255 116,256 117,256 117,256 117,256 117,257 117,259 118,259 118,259 118,262 118,262 119,262 119,263 119,263 119,263 119,263 120,261 120,261 120,261 120,262 121,262 121,262 121,262 121,262 121,263 122,262 122,263 122,264 122,263 123,264 123,264 123,263 123,257 123,255 124,255 124,254 124,256 124,257 125,257 125,257 125,258 125,256 125,258 126,253 126,255 126,256 126,257 127,258 127,256 127,254 127,254 127,253 128,253 128,253 128,253 128,252 129,251 129,251 129,253 129,251 129,253 130,254 130,253 130,252 130,254 131,253 131,254 131,255 131,256 131,254 132,256 132,256 132,255 132,255 133,255 133,256 133,255 133,255 134,255 134,256 134,255 134,252 134,253 135,252 135,250 135,251 135,251 136,250 136,247 136,248 136,249 136,246 137,246 137,246 137,246 137,244 138,244 138,242 138,243 138,242 138,246 139,247 139,246 139,247 139,247 140,247 140,247 140,245 140,245 140,245 141,245 141,244 141,243 141,242 142,242 142,242 142,242 142,240 142,240 143,241 143,242 143,244 143,244 144,244 144,246 144,247 144,247 144,248 145,248 145,247 145,241 145,234 146,234 146,235 146,237 146,239 146,238 147,236 147,240 147,242 147,242 148,244 148,247 148,246 148,245 148,246 149,243 149,239 149,240 149,240 150,238 150,241 150,243 150,244 150,241 151,242 151,241 151,240 151,240 152,240 152,246 152,246 152,246 153,242 153,243 153,242 153,242 153,242 154,241 154,239 154,238 154,236 155,236 155,237 155,237 155,237 155,237 156,237 156,234 156,235 156,233 157,234 157,233 157,230 157,230 157,231 158,229 158,228 158,229 158,229 159,229 159,229 159,228 159,228 159,228 160,229 160,230 160,227 160,227 161,227 161,227 161,227 161,227 161,225 162,223 162,224 162,223 162,224 163,225 163,225 163,224 163,225 163,223 164,224 164,226 164,226 164,226 165,228 165,227 165,227 165,227 165,227 166,227 166,226 166,228 166,226 167,228 167,229 167,229 167,231 167,231 168,231 168,232 168,229 168,229 169,227 169,223 169,224 169,222 169,224 170,223 170,223 170,216 170,217 171,216 171,214 171,214 171,214 172,214 172,216 172,215 172,215 172,213 173,206 173,200 173,200 173,194 174,193 174,194 174,196 174,194 174,197 175,200 175,201 175,204 175,193 176,193 176,192 176,194 176,194 176,192 177,189 177,181 177,180 177,180 178,181 178,181 178,181 178,187 178,187 179,183 179,186 179,190 179,193 180,190 180,192 180,188 180,184 180,184 181,186 181,184 181,185 181,185 182,186 182,183 182,183 182,185 182,185 183,182 183,183 183,182 183,180 184,181 184,180 184,181 184,181 184,181 185,177 185,175 185,173 185,170 186,171 186,172 186,174 186,175 186,174 187,175 187,177 187,

In [8]:
//every strategy finally will be implement the trait `ApiType`, which accept `tick_data` and `hold`, and return an order action
// in live trading, these data comes from the trade api, so that every strategy can be put on live trading.
// To backtest on `Vec<TickData>` for more information of a strategy, a order matching method must be set to simulate 
// order matching on live trading.
let trade_info_vec = {
    let with_di_kline = WithDiKline { data: BtWrapper(two_ma_stra_ptm.clone()), di: &di };
    let with_algo_box = WithAlgoBox { data: with_di_kline, algo: Box::new(TargetSimple) }; //set the method of opening a order
    WithMatchBox { data: with_algo_box, match_box: Box::new(MatchMean) } // set the method of order matching
}.bt_tick(&tick_data);// backtest the strategy on tick data
trade_info_vec.evcxr_display();
TickerTradeInfo { ticker: aler, trade_info_vec }.into_pnl_res(); //convert the trade info to pnl

"TradeInfo { time: 2024-09-18T21:00:00.500, action: LoOpen(1, 20012.5) }"
"TradeInfo { time: 2024-09-19T09:13:00, action: ShClose(1, 19825.0) }"
"TradeInfo { time: 2024-09-19T10:33:00, action: LoOpen(1, 19945.0) }"
"TradeInfo { time: 2024-09-20T09:43:00.500, action: ShClose(1, 20030.0) }"
"TradeInfo { time: 2024-09-20T10:53:01.500, action: LoOpen(1, 20085.0) }"
"TradeInfo { time: 2024-09-20T14:03:00, action: ShClose(1, 20080.0) }"
"TradeInfo { time: 2024-09-20T14:43:01, action: LoOpen(1, 20090.0) }"
"TradeInfo { time: 2024-09-23T09:13:01.500, action: ShClose(1, 19835.0) }"
"TradeInfo { time: 2024-09-23T13:43:01, action: LoOpen(1, 19810.0) }"
"TradeInfo { time: 2024-09-23T14:13:01, action: ShClose(1, 19767.5) }"
"TradeInfo { time: 2024-09-24T09:23:00, action: LoOpen(1, 19900.0) }"


### **2. Build a strategy based on kline index input.**</br>
#### &emsp; But I just care about some conditions, when these conditions met, I open a positon or close a existed one.

In [9]:
#[ta_derive2]
pub struct TwoMaStraOpen(pub usize, pub usize); 


#[typetag::serde]
impl Cond for TwoMaStraOpen {
    fn cond<'a>(&self,di: &'a Di) -> LoopSig<'a> {
        let short_ma = di.c().roll(RollFunc::Mean, RollOps::N(self.0));
        let long_ma = di.c().roll(RollFunc::Mean, RollOps::N(self.1));
        Box::new(move |e, _| {
            e > 0 && short_ma[e - 1] < long_ma[e - 1] && short_ma[e] >= long_ma[e]
        })
    }
}

#[ta_derive2]
pub struct TwoMaStraExit(pub usize, pub usize); 


#[typetag::serde]
impl Cond for TwoMaStraExit {
    fn cond<'a>(&self,di: &'a Di) -> LoopSig<'a> {
        let short_ma = di.c().roll(RollFunc::Mean, RollOps::N(self.0));
        let long_ma = di.c().roll(RollFunc::Mean, RollOps::N(self.1));
        Box::new(move |e, _| {
            short_ma[e] < long_ma[e]
        })
    }
}
let two_ma_stra_bool: Ptm = (two_ma::TwoMaStraOpen(10, 20), two_ma::TwoMaStraExit(10, 20)).to_ptm();

In [10]:
two_ma_stra_bool
    .bt_kline((&di, cs2))
    .da()
    .plot()

<polyline fill="none" opacity="1" stroke="#FFFFFF" stroke-width="1" points="75,246 75,247 75,249 75,249 75,248 76,249 76,248 76,251 76,248 77,248 77,249 77,249 77,249 77,249 78,249 78,249 78,249 78,249 79,249 79,249 79,249 79,245 79,244 80,244 80,243 80,244 80,244 81,244 81,245 81,243 81,244 81,244 82,244 82,244 82,245 82,244 83,245 83,246 83,246 83,246 83,246 84,246 84,246 84,243 84,242 85,242 85,242 85,241 85,238 85,238 86,239 86,239 86,240 86,240 87,240 87,240 87,239 87,240 87,240 88,240 88,240 88,241 88,239 89,239 89,240 89,240 89,240 89,239 90,238 90,238 90,239 90,238 91,238 91,238 91,238 91,238 91,238 92,238 92,238 92,238 92,238 93,238 93,238 93,238 93,240 93,240 94,242 94,242 94,242 94,242 95,243 95,242 95,242 95,242 96,243 96,243 96,243 96,243 96,242 97,243 97,243 97,243 97,243 98,243 98,244 98,244 98,244 98,244 99,245 99,245 99,245 99,245 100,245 100,244 100,243 100,245 100,245 101,245 101,245 101,247 101,246 102,246 102,248 102,249 102,249 102,246 103,245 103,246 103,247 103,248 104,247 104,247 104,248 104,248 104,250 105,249 105,251 105,251 105,251 106,252 106,252 106,252 106,252 106,252 107,253 107,254 107,254 107,254 108,252 108,252 108,252 108,253 108,253 109,253 109,253 109,252 109,252 110,253 110,253 110,255 110,255 110,254 111,255 111,255 111,255 111,254 112,255 112,252 112,253 112,252 112,252 113,253 113,253 113,253 113,252 114,252 114,251 114,253 114,253 115,253 115,253 115,254 115,253 115,256 116,255 116,255 116,255 116,256 117,256 117,256 117,256 117,257 117,259 118,259 118,259 118,262 118,262 119,262 119,263 119,263 119,263 119,263 120,261 120,261 120,261 120,262 121,262 121,262 121,262 121,262 121,263 122,262 122,263 122,264 122,263 123,264 123,264 123,263 123,257 123,255 124,255 124,254 124,256 124,257 125,257 125,257 125,258 125,256 125,258 126,253 126,255 126,256 126,257 127,258 127,256 127,254 127,254 127,253 128,253 128,253 128,253 128,252 129,251 129,251 129,253 129,251 129,253 130,254 130,253 130,252 130,254 131,253 131,254 131,255 131,256 131,254 132,256 132,256 132,255 132,255 133,255 133,256 133,255 133,255 134,255 134,256 134,255 134,252 134,253 135,252 135,250 135,251 135,251 136,250 136,247 136,248 136,249 136,246 137,246 137,246 137,246 137,244 138,244 138,242 138,243 138,242 138,246 139,247 139,246 139,247 139,247 140,247 140,247 140,245 140,245 140,245 141,245 141,244 141,243 141,242 142,242 142,242 142,242 142,240 142,240 143,241 143,242 143,244 143,244 144,244 144,246 144,247 144,247 144,248 145,248 145,247 145,241 145,234 146,234 146,235 146,237 146,239 146,238 147,236 147,240 147,242 147,242 148,244 148,247 148,246 148,245 148,246 149,243 149,239 149,240 149,240 150,238 150,241 150,243 150,244 150,241 151,242 151,241 151,240 151,240 152,240 152,246 152,246 152,246 153,242 153,243 153,242 153,242 153,242 154,241 154,239 154,238 154,236 155,236 155,237 155,237 155,237 155,237 156,237 156,234 156,235 156,233 157,234 157,233 157,230 157,230 157,231 158,229 158,228 158,229 158,229 159,229 159,229 159,228 159,228 159,228 160,229 160,230 160,227 160,227 161,227 161,227 161,227 161,227 161,225 162,223 162,224 162,223 162,224 163,225 163,225 163,224 163,225 163,223 164,224 164,226 164,226 164,226 165,228 165,227 165,227 165,227 165,227 166,227 166,226 166,228 166,226 167,228 167,229 167,229 167,231 167,231 168,231 168,232 168,229 168,229 169,227 169,223 169,224 169,222 169,224 170,223 170,223 170,216 170,217 171,216 171,214 171,214 171,214 172,214 172,216 172,215 172,215 172,213 173,206 173,200 173,200 173,194 174,193 174,194 174,196 174,194 174,197 175,200 175,201 175,204 175,193 176,193 176,192 176,194 176,194 176,192 177,189 177,181 177,180 177,180 178,181 178,181 178,181 178,187 178,187 179,183 179,186 179,190 179,193 180,190 180,192 180,188 180,184 180,184 181,186 181,184 181,185 181,185 182,186 182,183 182,183 182,185 182,185 183,182 183,183 183,182 183,180 184,181 184,180 184,181 184,181 184,181 185,177 185,175 185,173 185,170 186,171 186,172 186,174 186,175 186,174 187,175 187,177 187,

### **3. Build a strategy accepting tick data and current holding then return a order action**

In [11]:
#[ta_derive2]
pub struct TwoMaTickOrderAction;

impl ApiType for TwoMaTickOrderAction {
    fn api_type(&self) -> RetFnApi {
        let mut short_ma = SMA::new(1200).unwrap();
        let mut long_ma = SMA::new(2400).unwrap();
        let mut last_short_value = 0f64;
        let mut last_long_value = 0f64;
        Box::new(move |stream_api| {
            let c = stream_api.tick_data.c as f64;
            let short_value = short_ma.next(c);
            let long_value = long_ma.next(c);
            let hold = stream_api.hold.sum();
            let mut res = OrderAction::No;
            if hold == 0 {
                match last_short_value != 0. && last_short_value < last_long_value && short_value >= long_value {
                    true => {
                        res = OrderAction::LoOpen(1, stream_api.tick_data.bid1);
                    }
                    false => (),
                }
            } else if hold > 0 && short_value < long_value {
                res = OrderAction::ShClose(hold, stream_api.tick_data.ask1);
            }
            last_short_value = short_value;
            last_long_value = long_value;
            res
        })
    }
}

In [12]:
// because this stra implement on `ApiType`, so it can put on live trading directly, and when backtest on 
// tick data, there is no need to set the algo method, only order match method.
let trade_info_vec: Vec<TradeInfo> = WithMatchBox {
    data: TwoMaTickOrderAction,
    match_box: Box::new(MatchSimple),
}
    .bt_tick(&tick_data);

In [13]:
TickerTradeInfo {
    ticker: aler,
    trade_info_vec: trade_info_vec.clone() 
}.into_pnl_res().plot()

0
 
 
 
225
 
 
 
450
 
 
 
675
 
 
 
900
 
 
 
 
2024-09-18
 
 
 
2024-09-19
 
 
 
2024-09-20
 
 
 
2024-09-23
 
 
 
2024-09-24

### **4. Build a stragety accepting both tick data, kline and current holding, then return a order action**
#### &emsp; In some situation, I build a strategy basing on kline input, but I also need to make decisions </br> &emsp; base on tick data rather than wait until this kline finished.

In [14]:
#[ta_derive2]
pub struct MaTa(pub usize, pub usize);

#[typetag::serde]
impl Ta for MaTa {
    fn calc_da(&self,da:Vec< &[f32]> ,_di: &Di) -> vv32 {
        let short_ma = da[0].roll(RollFunc::Mean, RollOps::N(self.0));
        let long_ma = da[0].roll(RollFunc::Mean, RollOps::N(self.1));
        vec![short_ma, long_ma]
    }
}

#[typetag::serde]
impl CondType4 for TwoMaTickOrderAction {
    fn cond_type4(&self, _di: &Di) -> RetFnCondType4 {
        let ta_ops = MaTa(10, 20);
        Box::new(move |stream_cond_type1| {
            let ma_ta_res = stream_cond_type1.di_kline_state.di_kline.di.calc(&ta_ops);
            let tick_data = stream_cond_type1.stream_api.tick_data;
            let i = stream_cond_type1.di_kline_state.di_kline.i;
            let hold = stream_cond_type1.stream_api.hold.sum();
            let mut res = OrderAction::No;
            if hold == 0 {
                if tick_data.c > ma_ta_res[0][i] && ma_ta_res[0][i-1] < ma_ta_res[1][i-1] {
                    res = OrderAction::LoOpen(1, tick_data.bid1);
                }
            } else if hold > 0 && tick_data.c < ma_ta_res[0][i] {
                res = OrderAction::ShClose(hold, tick_data.ask1);
            }
            res
        })
    }
}

In [15]:
// because this stra accepts both tick data and kline, so when backtest on tick data, the input is `(&di, &tick_data)`
let data: CondType4Box = Box::new(two_ma::TwoMaTickOrderAction);
let trade_info_vec: Vec<TradeInfo> = WithMatchBox {
    data,
    match_box: Box::new(MatchMean),
}.bt_tick((&di, &tick_data));

In [16]:
TickerTradeInfo {
    ticker: aler,
    trade_info_vec: trade_info_vec.clone() 
}.into_pnl_res().plot()

<polyline fill="none" opacity="1" stroke="#FFFFFF" stroke-width="1" points="75,26 77,29 79,29 81,32 83,32 86,35 88,35 90,5 92,6 95,9 97,9 99,12 101,12 104,17 106,18 108,20 110,21 113,22 115,23 117,24 119,24 121,29 124,29 126,33 128,33 130,35 133,35 135,32 137,32 139,13 142,14 144,27 146,27 148,28 151,29 153,31 155,32 157,35 160,35 162,38 164,38 166,40 168,40 171,41 173,42 175,45 177,45 180,48 182,48 184,51 186,51 189,54 191,54 193,57 195,57 198,54 200,54 202,57 204,57 206,60 209,60 211,63 213,63 215,62 218,63 220,66 222,66 224,74 227,74 229,77 231,77 233,80 236,80 238,83 240,83 242,85 245,85 247,87 249,87 251,90 253,90 256,93 258,93 260,95 262,95 265,97 267,97 269,100 271,100 274,103 276,103 278,106 280,106 283,114 285,114 287,117 289,118 291,115 294,116 296,117 298,118 300,120 303,121 305,122 307,122 309,124 312,124 314,126 316,126 318,128 321,128 323,130 325,130 327,131 330,132 332,135 334,135 336,138 338,138 341,140 343,140 345,141 347,142 350,145 352,145 354,146 356,147 359,149 361,150 363,151 365,152 368,153 370,153 372,156 374,157 377,158 379,158 381,161 383,162 385,163 388,163 390,166 392,167 394,167 397,167 399,169 401,169 403,172 406,172 408,175 410,175 412,178 415,178 417,180 419,180 421,182 423,182 426,185 428,185 430,188 432,188 435,190 437,190 439,192 441,192 444,194 446,194 448,197 450,197 453,198 455,199 457,202 459,202 462,205 464,205 466,205 468,206 470,208 473,209 475,212 477,212 479,213 482,214 484,216 486,217 488,218 491,219 493,221 495,222 497,225 500,225 502,226 504,227 506,230 508,230 511,230 513,230 515,233 517,234 520,235 522,235 524,237 526,237 529,239 531,239 533,241 535,241 538,243 540,243 542,243 544,243 547,245 549,245 551,247 553,247 555,247 558,248 560,249 562,250 564,251 567,251 569,252 571,252 573,255 576,255 578,257 580,257 582,260 585,260 587,264 589,264 591,218 "/>
 
 
0k
 
 
 
 
2024-09-19
 
 
 
2024-09-20
 
 
 
2024-09-23
 
 
 
2024-09-24

### **5. There are other ways to construct a strategy**
### &emsp; Any way can be implemented to construct a strategy, as long as it finnally implement the trait `ApiType`,</br> &emsp; so that this strategy can be put on live trading.

In [17]:
//It's easy to backtest serveral strategies to the same data set in paralles
let pnl_res_double1: Vec<PnlRes<dt>> = vec![two_ma_stra_ptm.clone(), two_ma_stra_bool.clone()]
    .bt_kline((&di, cs2));

In [18]:
pnl_res_double1.iter().map(|x| x.da()).collect_vec().aplot(2)

<polyline fill="none" opacity="1" stroke="#FFFFFF" stroke-width="1" points="75,199 75,201 75,202 75,202 75,201 75,202 75,202 76,204 76,201 76,202 76,202 76,202 76,202 77,202 77,202 77,202 77,202 77,202 77,202 78,202 78,202 78,199 78,198 78,198 78,197 78,198 79,198 79,198 79,199 79,198 79,198 79,198 80,198 80,198 80,199 80,198 80,199 80,199 81,199 81,200 81,200 81,200 81,200 81,197 81,196 82,196 82,196 82,196 82,193 82,193 82,194 83,194 83,194 83,195 83,194 83,195 83,194 84,194 84,195 84,195 84,195 84,195 84,194 85,194 85,195 85,195 85,195 85,194 85,193 85,193 86,194 86,193 86,193 86,193 86,193 86,193 87,193 87,193 87,193 87,193 87,193 87,193 88,193 88,193 88,195 88,195 88,196 88,196 88,196 89,197 89,197 89,196 89,197 89,197 89,197 90,197 90,197 90,197 90,197 90,197 90,197 91,197 91,198 91,198 91,198 91,198 91,198 92,198 92,198 92,198 92,199 92,199 92,199 92,198 93,197 93,198 93,198 93,199 93,199 93,200 94,200 94,200 94,201 94,202 94,202 94,200 95,199 95,200 95,200 95,201 95,200 95,201 95,201 96,201 96,203 96,202 96,204 96,204 96,203 97,204 97,204 97,205 97,205 97,205 97,205 98,206 98,206 98,206 98,205 98,204 98,205 98,205 99,205 99,205 99,205 99,204 99,204 99,205 100,206 100,207 100,207 100,206 100,207 100,207 101,207 101,206 101,207 101,205 101,205 101,205 102,204 102,205 102,205 102,205 102,204 102,204 102,204 103,205 103,205 103,205 103,206 103,206 103,206 104,207 104,206 104,207 104,207 104,208 104,208 105,208 105,208 105,208 105,210 105,210 105,210 105,213 106,213 106,213 106,213 106,213 106,213 106,213 107,212 107,211 107,212 107,213 107,213 107,213 108,213 108,212 108,213 108,213 108,213 108,214 109,213 109,214 109,214 109,213 109,208 109,207 109,207 110,206 110,208 110,209 110,208 110,208 110,209 111,208 111,209 111,205 111,207 111,208 111,208 112,209 112,207 112,206 112,206 112,205 112,206 112,205 113,205 113,204 113,204 113,204 113,205 113,204 114,205 114,206 114,205 114,204 114,206 114,205 115,206 115,207 115,207 115,206 115,208 115,208 116,207 116,207 116,207 116,207 116,207 116,207 116,207 117,207 117,207 117,204 117,205 117,204 117,203 118,203 118,204 118,202 118,201 118,201 118,202 119,200 119,200 119,200 119,200 119,198 119,198 119,196 120,197 120,196 120,200 120,200 120,200 120,200 121,200 121,201 121,200 121,198 121,199 121,199 122,199 122,198 122,197 122,196 122,196 122,196 122,196 123,195 123,195 123,196 123,196 123,198 123,198 124,198 124,199 124,200 124,201 124,201 124,201 125,200 125,195 125,190 125,190 125,191 125,192 126,194 126,193 126,192 126,195 126,196 126,197 126,198 127,201 127,200 127,198 127,199 127,197 127,194 128,194 128,195 128,193 128,196 128,197 128,198 129,195 129,196 129,195 129,195 129,195 129,195 129,199 130,199 130,200 130,197 130,197 130,197 130,197 131,196 131,195 131,194 131,193 131,191 131,192 132,192 132,192 132,193 132,193 132,192 132,190 133,191 133,189 133,190 133,189 133,187 133,187 133,187 134,186 134,185 134,186 134,186 134,186 134,186 135,185 135,185 135,185 135,186 135,187 135,184 136,184 136,185 136,184 136,184 136,185 136,183 136,181 137,182 137,181 137,182 137,182 137,183 137,182 138,183 138,181 138,182 138,184 138,183 138,184 139,185 139,184 139,184 139,184 139,184 139,184 139,183 140,185 140,184 140,185 140,186 140,186 140,188 141,187 141,188 141,188 141,186 141,186 141,184 142,181 142,182 142,180 142,182 142,181 142,181 143,175 143,176 143,176 143,174 143,174 143,174 143,174 144,175 144,175 144,175 144,173 144,167 144,162 145,162 145,158 145,157 145,158 145,159 145,158 146,160 146,162 146,163 146,165 146,157 146,157 146,156 147,157 147,157 147,156 147,154 147,147 147,146 148,146 148,147 148,147 148,147 148,152 148,152 149,149 149,151 149,154 149,156 149,155 149,156 150,153 150,150 150,150 150,151 150,150 150,150 150,150 151,151 151,148 151,148 151,150 151,150 151,148 152,149 152,148 152,146 152,147 152,146 152,147 153,147 153,147 153,144 153,142 153,141 153,138 153,139 154,140 154,141 154,142 154,142 154,143 154,144 155,144 155,146 155,147 155,148 155,148 155,147 15

In [19]:
//orbacktest the same strategy on serveral data set
let pnl_res_double2: Vec<PnlRes<dt>> = two_ma_stra_ptm
   .bt_kline((vec![&di, &di], cs2));

In [20]:
pnl_res_double2.iter().map(|x| x.da()).collect_vec().aplot(2)

<polyline fill="none" opacity="1" stroke="#FFFFFF" stroke-width="1" points="75,199 75,201 75,202 75,202 75,201 75,202 75,202 76,204 76,201 76,202 76,202 76,202 76,202 77,202 77,202 77,202 77,202 77,202 77,202 78,202 78,202 78,199 78,198 78,198 78,197 78,198 79,198 79,198 79,199 79,198 79,198 79,198 80,198 80,198 80,199 80,198 80,199 80,199 81,199 81,200 81,200 81,200 81,200 81,197 81,196 82,196 82,196 82,196 82,193 82,193 82,194 83,194 83,194 83,195 83,194 83,195 83,194 84,194 84,195 84,195 84,195 84,195 84,194 85,194 85,195 85,195 85,195 85,194 85,193 85,193 86,194 86,193 86,193 86,193 86,193 86,193 87,193 87,193 87,193 87,193 87,193 87,193 88,193 88,193 88,195 88,195 88,196 88,196 88,196 89,197 89,197 89,196 89,197 89,197 89,197 90,197 90,197 90,197 90,197 90,197 90,197 91,197 91,198 91,198 91,198 91,198 91,198 92,198 92,198 92,198 92,199 92,199 92,199 92,198 93,197 93,198 93,198 93,199 93,199 93,200 94,200 94,200 94,201 94,202 94,202 94,200 95,199 95,200 95,200 95,201 95,200 95,201 95,201 96,201 96,203 96,202 96,204 96,204 96,203 97,204 97,204 97,205 97,205 97,205 97,205 98,206 98,206 98,206 98,205 98,204 98,205 98,205 99,205 99,205 99,205 99,204 99,204 99,205 100,206 100,207 100,207 100,206 100,207 100,207 101,207 101,206 101,207 101,205 101,205 101,205 102,204 102,205 102,205 102,205 102,204 102,204 102,204 103,205 103,205 103,205 103,206 103,206 103,206 104,207 104,206 104,207 104,207 104,208 104,208 105,208 105,208 105,208 105,210 105,210 105,210 105,213 106,213 106,213 106,213 106,213 106,213 106,213 107,212 107,211 107,212 107,213 107,213 107,213 108,213 108,212 108,213 108,213 108,213 108,214 109,213 109,214 109,214 109,213 109,208 109,207 109,207 110,206 110,208 110,209 110,208 110,208 110,209 111,208 111,209 111,205 111,207 111,208 111,208 112,209 112,207 112,206 112,206 112,205 112,206 112,205 113,205 113,204 113,204 113,204 113,205 113,204 114,205 114,206 114,205 114,204 114,206 114,205 115,206 115,207 115,207 115,206 115,208 115,208 116,207 116,207 116,207 116,207 116,207 116,207 116,207 117,207 117,207 117,204 117,205 117,204 117,203 118,203 118,204 118,202 118,201 118,201 118,202 119,200 119,200 119,200 119,200 119,198 119,198 119,196 120,197 120,196 120,200 120,200 120,200 120,200 121,200 121,201 121,200 121,198 121,199 121,199 122,199 122,198 122,197 122,196 122,196 122,196 122,196 123,195 123,195 123,196 123,196 123,198 123,198 124,198 124,199 124,200 124,201 124,201 124,201 125,200 125,195 125,190 125,190 125,191 125,192 126,194 126,193 126,192 126,195 126,196 126,197 126,198 127,201 127,200 127,198 127,199 127,197 127,194 128,194 128,195 128,193 128,196 128,197 128,198 129,195 129,196 129,195 129,195 129,195 129,195 129,199 130,199 130,200 130,197 130,197 130,197 130,197 131,196 131,195 131,194 131,193 131,191 131,192 132,192 132,192 132,193 132,193 132,192 132,190 133,191 133,189 133,190 133,189 133,187 133,187 133,187 134,186 134,185 134,186 134,186 134,186 134,186 135,185 135,185 135,185 135,186 135,187 135,184 136,184 136,185 136,184 136,184 136,185 136,183 136,181 137,182 137,181 137,182 137,182 137,183 137,182 138,183 138,181 138,182 138,184 138,183 138,184 139,185 139,184 139,184 139,184 139,184 139,184 139,183 140,185 140,184 140,185 140,186 140,186 140,188 141,187 141,188 141,188 141,186 141,186 141,184 142,181 142,182 142,180 142,182 142,181 142,181 143,175 143,176 143,176 143,174 143,174 143,174 143,174 144,175 144,175 144,175 144,173 144,167 144,162 145,162 145,158 145,157 145,158 145,159 145,158 146,160 146,162 146,163 146,165 146,157 146,157 146,156 147,157 147,157 147,156 147,154 147,147 147,146 148,146 148,147 148,147 148,147 148,152 148,152 149,149 149,151 149,154 149,156 149,155 149,156 150,153 150,150 150,150 150,151 150,150 150,150 150,150 151,151 151,148 151,148 151,150 151,150 151,148 152,149 152,148 152,146 152,147 152,146 152,147 153,147 153,147 153,144 153,142 153,141 153,138 153,139 154,140 154,141 154,142 154,142 154,143 154,144 155,144 155,146 155,147 155,148 155,148 155,147 15

## Di
### &emsp; `Di` can be viewed as a `Vec<KlineData>`, but for performance consideration, it has some other functions

### **1. Kline Data Container**

In [24]:
// di.to_csv("di.csv"); write the data to a csv
di

al - Rl5m       ---  1970-01-01 00:00:00      .. 2024-09-24 14:56:59.500   ---  99830      --- 0

In [25]:
di.get_part(NLast::Num(10))

al - Rl5m       ---  1970-01-01 00:00:00      .. 2024-09-24 14:56:59.500   ---  10         --- 0

In [26]:
for i in 0..5 {
    DiKline { di: &di, i }.print();
};

KlineData { t: 2015-01-05T09:18:00, o: 12970.0, h: 12980.0, l: 12965.0, c: 12965.0, v: 1612.0, ki: KlineInfo { open_time: 1970-01-01T00:00:00, pass_last: 0, pass_this: 0, contract: 0 } }
KlineData { t: 2015-01-05T09:23:00, o: 12965.0, h: 12970.0, l: 12950.0, c: 12955.0, v: 1738.0, ki: KlineInfo { open_time: 1970-01-01T00:00:00, pass_last: 0, pass_this: 0, contract: 0 } }
KlineData { t: 2015-01-05T09:28:01, o: 12950.0, h: 12955.0, l: 12935.0, c: 12935.0, v: 3328.0, ki: KlineInfo { open_time: 1970-01-01T00:00:00, pass_last: 0, pass_this: 0, contract: 0 } }
KlineData { t: 2015-01-05T09:33:08, o: 12935.0, h: 12945.0, l: 12925.0, c: 12930.0, v: 2702.0, ki: KlineInfo { open_time: 1970-01-01T00:00:00, pass_last: 0, pass_this: 0, contract: 0 } }
KlineData { t: 2015-01-05T09:38:00, o: 12930.0, h: 12930.0, l: 12920.0, c: 12930.0, v: 2310.0, ki: KlineInfo { open_time: 1970-01-01T00:00:00, pass_last: 0, pass_this: 0, contract: 0 } }
KlineData { t: 2015-01-05T09:23:00, o: 12965.0, h: 12970.0, l: 12

### **2. Has infomation**

In [27]:
//contract of  this data
di.pcon.ticker

al

In [28]:
// kline contruction method of this data
di.pcon.inter

Rl5m

In [29]:
Rl5m.intervals()[..3]

"Time(09:10:00, 09:12:59.500)"
"Time(09:13:00, 09:17:59.500)"
"Time(09:18:00, 09:22:59.500)"


In [30]:
// `di.pcon.inter` is the method to generate kline data from flow of tick data
// for example, I build a new inter:
#[derive(Debug, Clone, Serialize, Deserialize)]
pub struct TwoKlineOneDay;

#[typetag::serde]
impl Inter for TwoKlineOneDay {
    fn intervals(&self) -> Vec<Interval> {
        vec![
            Interval::Time(090000.to_tt(), 112959.5.to_tt()),
            Interval::Time(130000.to_tt(), 145959.5.to_tt())
        ]
    }
}

let two_kline_inter: InterBox = Box::new(TwoKlineOneDay);


//then I can get a `Di` for tick data:
let di_two_kline_one_day: Di = PriceTick::from_tick_data(&tick_data)
    .to_price_ori(two_kline_inter.tri_box(), aler)
    .to_di(aler, two_kline_inter.tri_box());

In [31]:
di_two_kline_one_day

al - TwoKlineOneDay ---  2024-09-19 09:00:00.500  .. 2024-09-24 15:00:00       ---  8          --- 82

In [32]:
for i in 0..3 {
    DiKline { di: &di_two_kline_one_day, i }.print();
};

KlineData { t: 2024-09-19T11:29:59.500, o: 19945.0, h: 19990.0, l: 19805.0, c: 19975.0, v: 77750.0, ki: KlineInfo { open_time: 2024-09-19T09:00:00.500, pass_last: 15513, pass_this: 12498, contract: 1 } }
KlineData { t: 2024-09-19T14:59:59.500, o: 19990.0, h: 20135.0, l: 19960.0, c: 20130.0, v: 49998.0, ki: KlineInfo { open_time: 2024-09-19T13:30:00.500, pass_last: 2, pass_this: 8704, contract: 1 } }
KlineData { t: 2024-09-20T11:29:59.500, o: 20085.0, h: 20115.0, l: 20000.0, c: 20080.0, v: 53328.0, ki: KlineInfo { open_time: 2024-09-20T09:00:00.500, pass_last: 14510, pass_this: 11492, contract: 1 } }
KlineData { t: 2024-09-19T14:59:59.500, o: 19990.0, h: 20135.0, l: 19960.0, c: 20130.0, v: 49998.0, ki: KlineInfo { open_time: 2024-09-19T13:30:00.500, pass_last: 2, pass_this: 8704, contract: 1 } }
KlineData { t: 2024-09-20T11:29:59.500, o: 20085.0, h: 20115.0, l: 20000.0, c: 20080.0, v: 53328.0, ki: KlineInfo { open_time: 2024-09-20T09:00:00.500, pass_last: 14510, pass_this: 11492, contra

In [33]:
// Or I can generate kline data from flow of kline data, because in some situations, bigger and smaller intervals of kline data
// would be used by mixed
let di_two_kline_one_day2: Di = di
    .calc(Event(two_kline_inter.pri_box()))
    .to_price_ori()
    .to_di(aler, two_kline_inter.tri_box());

In [34]:
di_two_kline_one_day2

al - TwoKlineOneDay ---  1970-01-01 00:00:00      .. 2024-09-24 13:32:59.500   ---  4643       --- 0

#### The trait `Tri` is the method to generate a kline data container from a flow of tick data,It's not limited to time intervals, it also can based on volatility threhold, order flow event or any other way.
This is an example:
```rust
#[ta_derive2]
pub struct OrderFlowPrice3(pub InterBox);

#[typetag::serde(name = "OrderFlowPrice3")]
impl Tri for OrderFlowPrice3 {
    fn update_tick_func(&self, ticker: Ticker) -> UpdateFuncTick {
        let mut kline = KlineStateInter::from_intervals(self.0.intervals());
        let mut last_info: bm<i32, (f32, f32)> = Default::default();
        let mut last_ask = f32::NAN;
        let mut last_bid = f32::NAN;
        let f_jump_time = inter::exit_by_jump_time.get_func(ticker);
        let f_last_time = inter::exit_by_last_time(0, 100).get_func(&self.0);
        Box::new(move |tick_data, price_ori| {
            let h = tick_data.t.hour();
            let of_triggered = (tick_data.c > last_ask  || tick_data.c < last_bid ) && h != 20 && h != 8 && tick_data.v != 0.;
            if of_triggered {
                let vv = last_info.entry((tick_data.c * 100.) as i32).or_default();
                if tick_data.c > last_ask {
                    vv.1 += tick_data.v;
                } else if tick_data.c < last_bid {
                    vv.0 += tick_data.v;
                }
            }
            let finish_triggered = of_triggered || f_jump_time(&tick_data.t.time()) || f_last_time(&tick_data.t);
            let time_kline_update = matches!(kline.kline_state.last, KlineState::Finished | KlineState::Ignor);
            if finish_triggered || time_kline_update {
                kline.update(tick_data);
                match kline.kline_state.last {
                    KlineState::Ignor => last_info.clear(),
                    KlineState::Finished => {
                        let interval_info = last_info
                            .iter()
                            .fold(init_a_matrix(last_info.len(), 3), |mut accu, (k, v)| {
                                accu[0].push(*k as f32 / 100.);
                                accu[1].push(v.0);
                                accu[2].push(v.1);
                                accu
                            });
                        price_ori.update(&kline.kline_state.data);
                        price_ori.immut_info.push(interval_info);
                        last_info.clear();
                    }
                    _ => {}
                }
            }
            last_ask = tick_data.ask1;
            last_bid = tick_data.bid1;
            kline.kline_state.last.clone()
        })
    }
}
```

### **3. Indicator calculation**
### &emsp; A `Di` can be shared by many strategies, and these strategies sometimes need to calculate the same incaditor. So once a indicator (implement on trait `Ta`) is calculated, the result will be cached in itself, and be cleared until a new kline updated.

In [35]:
di.calc(Rsi(10))[0][..10]

NaN
0.0
0.0
0.0
0.0
0.0
37.017292
43.54765
34.58325
27.505615


In [36]:
di.clear();
t!(di.calc(Atr(5));); // the first time to calc `Atr`
t!(di.calc(Atr(5));); // the second time to calc `Atr`, because it cached, so the time comsumed is less than the first one

1.380486ms
3.26µs
3.26µs


In [37]:
di.calc(Event(two_kline_inter.pri_box()) + ono + Atr(5)); // first convert the kline to another kline, and calculate
//`Atr` on the new kline

## Live trading

### Types implementing the trait `ApiType` can be used for live trading. `ApiType` accepts two inputs: `TickData` and `Hold`. These data are sent by the exchange API, such as market data returns and order action returns. After receiving these data, they are saved in a `Queue` to prevent data loss. Meanwhile, the strategy continuously sends `OrderAction` to the exchange API.</br> To implement an exchange API, the type must be able to receive `Vec<(TickData, Hold, OrderAction)>`. Whenever `TickData` or `Hold` is updated on the exchange side, it notifies the `ApiType` and sends the updated data back to `ApiType`. Meanwhile, the exchange API continuously accepts notifications of `OrderAction` from `ApiType`, and then handle the `OrderAction`.</br> The code from [`qust-stra/src/main`](https://github.com/baiguoname/qust/blob/main/qust-stra/src/main.rs) give an example to put a strategy on live trading(with excahgne API `CTP`).